In [1]:
import os
# These are the default AWS configurations for the Analysis Sandbox.
# that are set in the environmnet variables. 
aws_default_config = {
    #'AWS_NO_SIGN_REQUEST': 'YES', 
    'AWS_SECRET_ACCESS_KEY': 'fake',
    'AWS_ACCESS_KEY_ID': 'fake',
}

# To access public bucket, need to remove the AWS credentials in 
# the environment variables or the following error will occur.
# PermissionError: The AWS Access Key Id you provided does not exist in our records.

for key in aws_default_config.keys():
    if key in os.environ:
        del os.environ[key]

In [2]:
import json
import logging
import os
import collections

import click
import fsspec
import geopandas as gpd
import numpy as np
import rasterio.features
from datacube.utils.cog import to_cog
from datacube.utils.dask import save_blob_to_file, save_blob_to_s3
from datacube.utils.geometry import Geometry
from odc.dscache.tools.tiling import parse_gridspec_with_name
from odc.geo.geobox import GeoBox
from odc.geo.xr import wrap_xr
from pandas.api.types import is_float_dtype, is_integer_dtype, is_string_dtype

from deafrica_conflux.cli.logs import logging_setup
from deafrica_conflux.filter_polygons import get_intersecting_polygons
from deafrica_conflux.id_field import guess_id_field
from deafrica_conflux.io import check_dir_exists, check_file_exists, check_if_s3_uri

In [3]:
verbose = 1 #1
# Grid name africa_{10|20|30|60}
grid_name = "africa_30"
# Datacube product to get tiles for from the grid.
product = "wofs_ls"
# Path to the polygons to be rasterised.
polygons_file_path = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/historical_extent/waterbodies.parquet"
# Unique key id in polygons vector file
numeric_id = "WB_ID"
string_id = "UID"
# Directory to write the tiled polygon rasters to.
output_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/"
# Overwrite existing polygons raster file
overwrite = True

In [4]:
# Set up logger.
logging_setup(verbose)
_log = logging.getLogger(__name__)

In [5]:
# Support pathlib Paths.
polygons_file_path = str(polygons_file_path)
output_directory = str(output_directory)

In [6]:
# Create the output directory if it does not exist.
is_s3 = check_if_s3_uri(output_directory)
if is_s3:
    fs = fsspec.filesystem("s3")
else:
    fs = fsspec.filesystem("file")

if not check_dir_exists(output_directory):
    fs.makedirs(output_directory, exist_ok=True)
    _log.info(f"Created directory {output_directory}")

tiles_output_directory = os.path.join(output_directory, "product_tiles")
if not check_dir_exists(tiles_output_directory):
    fs.makedirs(tiles_output_directory, exist_ok=True)
    _log.info(f"Created directory {tiles_output_directory}")

rasters_output_directory = os.path.join(output_directory, "historical_extent_rasters")
if not check_dir_exists(rasters_output_directory):
    fs.mkdirs(rasters_output_directory, exist_ok=True)
    _log.info(f"Created directory {output_directory}")

[2024-01-25 12:36:12,976] {credentials.py:611} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2024-01-25 12:36:13,201] {960524324.py:20} INFO - Created directory s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/


In [7]:
# Get the GridSpec.
grid, gridspec = parse_gridspec_with_name(grid_name)
_log.info(f"Using the grid {grid} with {gridspec}")

# From the GridSpec get the crs and resolution.
crs = gridspec.crs
resolution = abs(gridspec.resolution[0])

[2024-01-25 12:36:13,214] {141367464.py:3} INFO - Using the grid africa_30 with GridSpec(crs=epsg:6933, tile_size=(96000.0, 96000.0), resolution=(-30, 30))


In [8]:
# Read the product footprint.
product_footprint = gpd.read_file(
    f"https://explorer.digitalearth.africa/api/footprint/{product}"
).to_crs(crs)
# Get the product footprint geopolygon.
product_footprint = Geometry(geom=product_footprint.geometry[0], crs=crs)

In [9]:
# Get the tiles covering the product footprint.
tiles = gridspec.tiles_from_geopolygon(geopolygon=product_footprint)
tiles = list(tiles)

# Get the individual tile geometries.
tile_geometries = []
tile_ids = []
for tile in tiles:
    tile_idx, tile_idy = tile[0]
    tile_geometry = tile[1].extent.geom

    tile_geometries.append(tile_geometry)
    tile_ids.append(f"x{tile_idx:03d}_y{tile_idy:03d}")

tiles_gdf = gpd.GeoDataFrame(data={"tile_ids": tile_ids, "geometry": tile_geometries}, crs=crs)
_log.info(f"Tile count: {len(tiles_gdf)}")

tiles_output_fp = os.path.join(tiles_output_directory, f"{product}_tiles.parquet")
tiles_gdf.to_parquet(tiles_output_fp)
_log.info(f"{product} tiles written to {tiles_output_fp}")

[2024-01-25 12:36:16,844] {2670914241.py:16} INFO - Tile count: 6010
[2024-01-25 12:36:17,084] {2670914241.py:20} INFO - wofs_ls tiles written to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/product_tiles/wofs_ls_tiles.parquet


In [10]:
# Load the polygons.
try:
    polygons_gdf = gpd.read_parquet(polygons_file_path).to_crs(crs)
except Exception:
    _log.info("Polygons vector file is not a parquet file")
    try:
        polygons_gdf = gpd.read_file(polygons_file_path).to_crs(crs)
    except Exception as error:
        _log.error(f"Could not load file {polygons_file_path}")
        _log.error(error)
        raise error

_log.info(f"Polygon count {len(polygons_gdf)}")

[2024-01-25 12:36:17,473] {2615288541.py:13} INFO - Polygon count 8768


In [11]:
# Check the id columns are unique.
numeric_id = guess_id_field(input_gdf=polygons_gdf, use_id=numeric_id)
assert is_integer_dtype(polygons_gdf[numeric_id]) or is_float_dtype(
    polygons_gdf[numeric_id]
)

string_id = guess_id_field(input_gdf=polygons_gdf, use_id=string_id)
assert is_string_dtype(polygons_gdf[string_id])

[2024-01-25 12:36:17,480] {id_field.py:64} INFO - Values in the column WB_ID are unique.
[2024-01-25 12:36:17,483] {id_field.py:64} INFO - Values in the column UID are unique.


In [12]:
polygon_numericids_to_stringids = dict(zip(polygons_gdf[numeric_id], polygons_gdf[string_id]))
polygon_numericids_to_stringids_fp = os.path.join(rasters_output_directory, "polygon_numericids_to_stringids.json")
with fs.open(polygon_numericids_to_stringids_fp, "w") as fp:
    json.dump(polygon_numericids_to_stringids, fp)
_log.info(f"Polygon numeric IDs (WB_ID) to string IDs (UID) dictionary written to {polygon_numericids_to_stringids_fp}")

[2024-01-25 12:36:17,701] {488506305.py:5} INFO - Polygon numeric IDs (WB_ID) to string IDs (UID) dictionary written to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/polygon_numericids_to_stringids.json


In [13]:
_log.info("Filtering out tiles that do not intersect with any polygon...")
filtered_tiles_gdf = get_intersecting_polygons(
    region=polygons_gdf, polygons_gdf=tiles_gdf, use_id="tile_ids"
)
_log.info(f"Filtered out {len(tiles_gdf) - len(filtered_tiles_gdf)} tiles.")
_log.info(f"Filtered tiles count: {len(filtered_tiles_gdf)}.")

[2024-01-25 12:36:17,706] {4261533379.py:1} INFO - Filtering out tiles that do not intersect with any polygon...
[2024-01-25 12:36:17,708] {id_field.py:64} INFO - Values in the column tile_ids are unique.
[2024-01-25 12:36:17,758] {4261533379.py:5} INFO - Filtered out 5950 tiles.
[2024-01-25 12:36:17,759] {4261533379.py:6} INFO - Filtered tiles count: 60.


In [14]:
# Split each row in the tiles into a GeoDataFrame of its own.
tiles = np.array_split(filtered_tiles_gdf, len(filtered_tiles_gdf))
assert len(tiles) == len(filtered_tiles_gdf)

polygons_stringids_to_tileids = collections.defaultdict(list)
for i, tile in enumerate(tiles):
    tile_id = tile["tile_ids"].iloc[0]
    tile_geometry = tile.geometry.iloc[0]
    _log.info(f"Rasterizing polygons for tile {tile_id} ({i + 1}/{len(tiles)})")

    tile_raster_fp = os.path.join(rasters_output_directory, f"{tile_id}.tif")

    if not overwrite:
        _log.info(f"Checking existence of {tile_raster_fp}")
        exists = check_file_exists(tile_raster_fp)

    if overwrite or not exists:
        # Get the geobox for the region.
        tile_geobox = GeoBox.from_geopolygon(
            geopolygon=Geometry(geom=tile_geometry, crs=crs),
            resolution=resolution,
            crs=crs,
        )

        # Get the polygons that intersect with the tile.
        tile_polygons = get_intersecting_polygons(
            region=tile, polygons_gdf=polygons_gdf, use_id=numeric_id
        )
        for poly_id in tile_polygons[string_id].to_list():
            polygons_stringids_to_tileids[poly_id].append(tile_id)

        # Rasterise shapes into a numpy array
        shapes = zip(tile_polygons.geometry, tile_polygons[numeric_id])
        tile_raster_np = rasterio.features.rasterize(
            shapes=shapes, out_shape=tile_geobox.shape, transform=tile_geobox.transform
        )

        # Convert numpy array to a full xarray.DataArray
        tile_raster_ds = wrap_xr(im=tile_raster_np, gbox=tile_geobox)

        # Write the raster to disk.
        cog_bytes = to_cog(tile_raster_ds)
        if check_if_s3_uri(tile_raster_fp):
            save_blob_to_s3(data=cog_bytes, url=tile_raster_fp).compute()
        else:
            save_blob_to_file(data=cog_bytes, url=tile_raster_fp).compute()
        _log.info(f"Exported raster data to {tile_raster_fp}")
    else:
        _log.info(f"{tile_raster_fp} already exists, skipping...")

[2024-01-25 12:36:17,864] {442804848.py:9} INFO - Rasterizing polygons for tile x168_y090 (1/60)
[2024-01-25 12:36:17,868] {id_field.py:64} INFO - Values in the column WB_ID are unique.


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/nump

[2024-01-25 12:36:18,380] {442804848.py:47} INFO - Exported raster data to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/x168_y090.tif
[2024-01-25 12:36:18,381] {442804848.py:9} INFO - Rasterizing polygons for tile x169_y090 (2/60)
[2024-01-25 12:36:18,383] {id_field.py:64} INFO - Values in the column WB_ID are unique.
[2024-01-25 12:36:18,674] {442804848.py:47} INFO - Exported raster data to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/x169_y090.tif
[2024-01-25 12:36:18,676] {442804848.py:9} INFO - Rasterizing polygons for tile x168_y091 (3/60)
[2024-01-25 12:36:18,677] {id_field.py:64} INFO - Values in the column WB_ID are unique.
[2024-01-25 12:36:18,959] {442804848.py:47} INFO - Exported raster data to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/x168_y091.tif
[2024-01-25 12:36:18,960] {442804848.py:9} INFO - Rasterizing polygons

In [15]:
polygons_stringids_to_tileids_fp = os.path.join(rasters_output_directory, "polygons_stringids_to_tileids.json")
with fs.open(polygons_stringids_to_tileids_fp, "w") as fp:
    json.dump(polygons_stringids_to_tileids, fp)
_log.info(f"Polygon string IDs (UID) to tile ids dictionary written to {polygons_stringids_to_tileids_fp}")

[2024-01-25 12:36:39,969] {294919685.py:4} INFO - Polygon string IDs (UID) to tile ids dictionary written to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/polygons_stringids_to_tileids.json
